In [2]:
from selenium import webdriver
from bs4 import BeautifulSoup as BS
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
import re
import time
from serde import serialize, deserialize
from serde.json import to_json, from_json
import pandas as pd
from requests import get 
from dataclasses import dataclass
from pays import Countries

In [3]:
@serialize
@deserialize
@dataclass
class Sejour :
    description : str
    prix : int
    lieux : str
    duree : int
    diff : str
    theme : str 

On récupère les backups réalisés précédemment afin de constituer une liste contenant tous les séjours. 

In [4]:
# Récupération du backup
with open("backup_terdav.json", "r") as fichier:
    data = fichier.read()

sejours = from_json(list[Sejour], data)

In [5]:
with open("backup_terdav2.json", "r") as fichier:
    data = fichier.read()

sejours2 = from_json(list[Sejour], data)

In [6]:
with open("backup_explora.json", "r") as fichier:
    data = fichier.read()

sejours3 = from_json(list[Sejour], data)

In [7]:
with open("backup_decathlon.json", "r") as fichier:
    data = fichier.read()

sejours4 = from_json(list[Sejour], data)

In [8]:
sejours_tot=sejours + sejours2 + sejours3 + sejours4

In [9]:
len(sejours_tot)

423

Nous n'avons malheureusement que 423 séjours. Il faudrait scraper d'autres sites afin d'avoir au moins 1000 séjours. 

Nous constituons un dataframe à partir de la liste des séjours.

In [10]:
columns = ['description', 'prix', 'lieux', 'duree', 'diff', 'theme']
df = pd.DataFrame(data=sejours_tot, columns= columns) 

Afin de faciliter la lecture et le travail sur le dataframe, nous affichons l'intégralité du dataframe

In [11]:
pd.set_option("max_rows", None)

In [60]:
df

,description,prix,lieux,duree,diff,theme
0,plaisirs neige &#224; sixt-fer-&#224;-cheval,690,alpes du nord,8,niveau 2,multi-activités
1,haute clar&#233;e &quot;espace trappeur&quot;,860,alpes du sud,7,niveau 1,multi-activités
2,aventures hivernales sur le plateau de l&#39;a...,720,massif central,6,niveau 1,multi-activités
3,neige et soleil du queyras en famille,730,alpes du sud,7,niveau 1,multi-activités
4,plaisirs neige dans les aravis,830,alpes du nord,7,niveau 1,multi-activités
5,vall&#233;e de la roy,670,alpes du sud,7,niveau 2,canyoning
6,davy crockett en haute bl&#233;one,730,alpes du sud,6,niveau 1,multi-activités
7,calanques et sensations proven&#231;ales,840,provence - côte d'azur,6,niveau 2,multi-activités
8,ea,760,massif central,7,niveau 1,multi-activités
9,aventures canyon en sierra de guara,820,pyrénées,7,niveau 1,canyoning


In [11]:
df.loc[0, "prix"]

690

Un rapide résumé du dataframe. La ligne `unique` nous intéresse particulièrement.

In [12]:
df.describe(include="all")

,description,prix,lieux,duree,diff,theme
count,423,423.000000,423,423.000000,423,423
unique,393,NaN,104,NaN,12,39
top,,NaN,France Auvergne Rhone Alpes,NaN,Découverte,Rando nature
freq,5,NaN,30,NaN,96,132
mean,NaN,863.456265,NaN,6.990544,NaN,NaN
std,NaN,751.312177,NaN,2.965825,NaN,NaN
min,NaN,0.000000,NaN,1.000000,NaN,NaN
25%,NaN,490.000000,NaN,6.000000,NaN,NaN
50%,NaN,680.000000,NaN,7.000000,NaN,NaN
75%,NaN,912.500000,NaN,8.000000,NaN,NaN


A l'aide de la commande `value_counts`, nous observons les différentes modalités des variable catégorielles. 

In [13]:
df["lieux"].value_counts()

France Auvergne Rhone Alpes          30
bretagne - normandie                 29
pyrénées                             28
massif central                       26
alpes du nord                        24
corse                                22
alpes du sud                         19
sud-ouest                            16
provence - côte d'azur               14
vallée de la loire                   13
                                     10
Suède Laponie                         9
autres régions                        8
France Provence Alpes Co...           7
Canaries/Espagne                      7
Norvège                               6
Cuba                                  6
Ecosse                                5
Irlande                               5
Jordanie                              5
Islande                               5
France                                5
Açores                                4
Madère/Portugal                       4
Corse                                 4


In [25]:
df["theme"].value_counts()

Rando nature          132
randonnée              78
vélo                   52
multi-activités        34
randonnée avec âne     18
Raquettes à neige      18
Ski de randonnée        9
Auberge                 8
Trek                    8
Chalet                  7
trek                    6
Randonnée               5
Ski nordique            4
autotour                4
Yoga                    4
Photographie            3
Alpinisme               3
Multiactivités été      3
Hôtel                   2
Couchette               2
découverte              2
canyoning               2
Cascade de glace        2
vtt                     2
micro-aventure          1
Logement privé          1
vélo de route           1
Surf house              1
Cabine                  1
Kite ski                1
Voile                   1
Cani rando              1
Canoë-kayak             1
Paddle                  1
Refuge                  1
Tente                   1
Escalade                1
raquette                1
Chiens de tr

In [76]:
df["diff"].value_counts()

Découverte    96
niveau 2      93
niveau 1      76
2             44
Initié        38
niveau 3      28
3             25
1              8
4              8
niveau 4       4
5              2
Confirmé       1
Name: diff, dtype: int64

On remarque qu'il y a beaucoup trop de modalités. 

Nous réalisons une copie du dataframe afin de pouvoir travailler dessus et éviter d'avoir à tout recharger chaque fois que nous faisons une erreur.

In [261]:
dftest = df.copy()

Après avoir observé les séjours sur les différents sites. IL faut choisir un classement pour standardiser (rendre compatibles) les modalités des variables des différents sites. 

On commence avec le niveau de difficulté. 

- Découverte = "niveau 1", "1", "2"
- Confirmé = "niveau 2", "3"
- Initié = "niveau 3", "niveau 4", "4", "5"

In [262]:
dftest.loc[(dftest["diff"] ==1)|(dftest["diff"] ==2)|(dftest["diff"] =="niveau 1"), "diff"] = "Découverte"
dftest.loc[(dftest["diff"] ==3)|(dftest["diff"] =="niveau 2"), "diff"] = "Initié"
dftest.loc[(dftest["diff"] ==4)|(dftest["diff"] ==5)|(dftest["diff"] == "niveau 3")|(dftest["diff"] == "niveau 4"), "diff"] = "Confirmé"

In [263]:
dftest["diff"].value_counts()

Découverte    224
Initié        156
Confirmé       43
Name: diff, dtype: int64

Ce regroupement nous semble pertinent.

Afin d'affiner le regroupement des thèmes, on regarde le détail des thèmes qu'on a du mal à classifier.
Un exemple ci dessous avec la modalit *Autres régions* de la variables **lieu**.

In [260]:
dftest[dftest["lieux"]=="autres régions"]

,description,prix,lieux,duree,diff,theme
53,aventures et nature au cœur du jura,0,autres régions,6,Initié,multi-activités
76,micro-aventures en alsace,410,autres régions,4,Initié,randonnée
92,escapade libert&#233; dans les vosges,520,autres régions,4,Confirmé,randonnée
139,ch&#226;teaux et vignobles d&#39;alsace,710,autres régions,7,Initié,randonnée
152,la grande travers&#233;e du jura en vtt &#233;...,750,autres régions,7,Initié,vtt
160,la vall&#233;e du doubs &#224; v&#233;lo,785,autres régions,7,Initié,vélo
170,villages et vignobles alsaciens &#224; v&#233;lo,815,autres régions,7,Initié,vélo
191,la viarh&#244;n,1750,autres régions,8,Découverte,vélo


On met toutes les variables catégorielles en minuscule afin de faciliter leur regroupement. En effet, on a certaines modalités qui ne diffèrent que par la première lettre en majuscule. 

In [264]:
dftest["theme"] = df["theme"].str.lower()
dftest["description"] = df["description"].str.lower()
dftest["lieux"] = df["lieux"].str.lower()

Le thème nous pose souvent problèmes. Certains sont classés en *hotel*, ou autre type de logements. On peut développer une fonction qui scan la description pour affiner le thème. Nous allons par exemple classer en activités nordique, tous les séjours qui comprennent le mot ski dans leur description. 

In [265]:
def theme_with_desc(df):
    for i in range(0, len(df)):
        if "ski" in df.iloc[i]["description"]:
            df.loc[i,"theme"] = "activités nordique"

theme_with_desc(dftest)

Cela fonctionne, mise à part qu'il semble y avoir du ski dans le désert...


In [97]:
dftest["theme"].value_counts()

rando nature          132
randonnée              83
vélo                   52
multi-activités        34
randonnée avec âne     18
raquettes à neige      18
trek                   14
ski de randonnée        9
auberge                 8
chalet                  7
ski nordique            4
yoga                    4
autotour                4
alpinisme               3
photographie            3
multiactivités été      3
cascade de glace        2
canyoning               2
découverte              2
couchette               2
vtt                     2
hôtel                   2
cani rando              1
raquette                1
escalade                1
tente                   1
refuge                  1
paddle                  1
canoë-kayak             1
chiens de traîneau      1
voile                   1
surf house              1
logement privé          1
micro-aventure          1
kite ski                1
vélo de route           1
cabine                  1
Name: theme, dtype: int64

- Rando nature = "Rando nature" "randonnée", "Trek", "Randonnée", "Auberge", "trek", "découverte", "Hôtel", "Refuge"
- Cyclo = "vélo", "vtt", "vélo de route"
- Multi-activités = "multi-activités", "Yoga", "autotour", "Photographie", "Multiactivités été", "canyoning", ...
- Rando avec animal = "randonnée avec un âne", "Chiens de traîneau", "Cani rando"
- Activités nordique = "Raquettes à neige", "Ski de radonnée", "Ski nordique", "Chalet", "Alpinisme", "Cascade de glace", "raquette"
- reste à trier = .... en multi activités

Pris par le temps, nous n'affinons pas plus en fonction de la description. Il aurait été par exemple possible de classer en multi-activités les séjours contenant *yoga* (ou autres activités) dans multi activités.

In [266]:
dftest.loc[(dftest["theme"] =="randonnée")|(dftest["theme"] =="trek")|(dftest["theme"] =="auberge")|(dftest["theme"] =="hôtel")|(dftest["theme"] =="refuge")|(dftest["theme"] =="découverte"), "theme"] = "rando nature"
dftest.loc[(dftest["theme"] == "vélo")|(dftest["theme"] == "vtt")|(dftest["theme"] == "vélo de route"), "theme"] = "cyclo"
dftest.loc[(dftest["theme"] == "randonnée avec âne")|(dftest["theme"] == "chiens de traîneau")|(dftest["theme"] == "cani rando"), "theme"] = "ani-rando"
dftest.loc[(dftest["theme"] =="raquettes à neige")|(dftest["theme"] =="ski de randonnée")|(dftest["theme"] =="ski nordique")|(dftest["theme"] =="chalet")|(dftest["theme"] =="alpinisme")|(dftest["theme"] =="cascade de glace")|(dftest["theme"] =="raquette")|(dftest["theme"] =="logement privé"), "theme"] = "activités nordique"

In [231]:
dftest["theme"].value_counts()

rando nature          241
cyclo                  55
activités nordique     50
multi-activités        34
ani-rando              20
autotour                4
yoga                    4
photographie            3
multiactivités été      3
canyoning               2
micro-aventure          1
surf house              1
voile                   1
canoë-kayak             1
paddle                  1
tente                   1
escalade                1
Name: theme, dtype: int64

Après un premier classement, nous constatons que les restants vont dans multi-activités. 

In [267]:
for i in range(0, len(dftest)):
    if "rando nature" not in dftest.iloc[i]["theme"] and "cyclo" not in dftest.iloc[i]["theme"] and "activités nordique" not in dftest.iloc[i]["theme"] and "multi-activités" not in dftest.iloc[i]["theme"] and "ani-rando" not in dftest.iloc[i]["theme"]: 
        dftest.loc[i,"theme"] = "multi-activités"

In [268]:
dftest["theme"].value_counts()

rando nature          241
multi-activités        57
cyclo                  55
activités nordique     50
ani-rando              20
Name: theme, dtype: int64

Ci dessus, le résultat du classement. 

Nous passons maintenant aux lieux. 

In [242]:
dftest["lieux"].value_counts()

metropole                            267
                                      10
suède laponie                          9
autres régions                         8
canaries/espagne                       7
cuba                                   6
norvège                                6
islande                                5
jordanie                               5
irlande                                5
ecosse                                 5
açores                                 4
madère/portugal                        4
srilanka                               4
grèce/lescyclades                      4
massifcentral/chemindecompostelle      3
patagonie                              3
suisse valais                          3
egypte/valléedunil                     3
turquie/lycie                          2
grèce/crête                            2
norvège                                2
italie/côteamalfitaine                 2
réunion                                2
finlande laponie

Classer la difficulté nous à sembler assez "naturels". Le classement des thèmes et des lieux nous semble plus problèmaique. N'allons nous pas perdre de l'information utile à la phase d'apprentissage? 

Concernant les lieux, nous décidons de classer en trois modalités : *métropole*, *pays proches*, *pays éloigné*.
Nous avons essayé, afin de ne pas créer les listes de tri nous même, d'utiliser des librairies de **pays**. Nous ne sommes pas parvenus à un résultat. Nous avons donc constituer les listes de classement nous mêmes. 
Nous pensons qu'utiliser des librairies (ou liste déjà faites), ou que d'autre moyens aurait été plus pertienent pour automatiser notre système de classement. 

Nous nous contentons finalement de tracer une zone géographique proche à l'aide de google map et nous créons la liste à partir de cela. 

In [269]:
region = ["auvergne", "rhone", "alpes", "pyrénées", "massif central", "sud-ouest", "provence", "bourogne", "bretagne", "normandie", "loire", "france", "occitanie", "normandes", "autres régions"]
pays_proche = ["madère", "maroc", "italie", "catalogne", "espagne", "grèce", "suisse", "ecosse", "irlande", "corse"]

In [270]:
for i in range(0, len(dftest)):
    for str in region:
        if str in dftest.loc[i,"lieux"]:
            dftest.loc[i, "lieux"] = "metropole"
    for str in pays_proche:
        if str in dftest.loc[i, "lieux"]:
            dftest.loc[i, "lieux"] = "dest_proche"

In [271]:
dftest["lieux"].value_counts()

dest_eloignee    423
Name: lieux, dtype: int64

In [12]:
# all code 

dftest = df.copy()
dftest.loc[(dftest["diff"] ==1)|(dftest["diff"] ==2)|(dftest["diff"] =="niveau 1"), "diff"] = "Découverte"
dftest.loc[(dftest["diff"] ==3)|(dftest["diff"] =="niveau 2"), "diff"] = "Initié"
dftest.loc[(dftest["diff"] ==4)|(dftest["diff"] ==5)|(dftest["diff"] == "niveau 3")|(dftest["diff"] == "niveau 4"), "diff"] = "Confirmé"
dftest["theme"] = df["theme"].str.lower()
dftest["description"] = df["description"].str.lower()
dftest["lieux"] = df["lieux"].str.lower()

def theme_with_desc(df):
    for i in range(0, len(df)):
        if "ski" in df.iloc[i]["description"]:
            df.loc[i,"theme"] = "activités nordique"

theme_with_desc(dftest)

dftest.loc[(dftest["theme"] =="randonnée")|(dftest["theme"] =="trek")|(dftest["theme"] =="auberge")|(dftest["theme"] =="hôtel")|(dftest["theme"] =="refuge")|(dftest["theme"] =="découverte"), "theme"] = "rando nature"
dftest.loc[(dftest["theme"] == "vélo")|(dftest["theme"] == "vtt")|(dftest["theme"] == "vélo de route"), "theme"] = "cyclo"
dftest.loc[(dftest["theme"] == "randonnée avec âne")|(dftest["theme"] == "chiens de traîneau")|(dftest["theme"] == "cani rando"), "theme"] = "ani-rando"
dftest.loc[(dftest["theme"] =="raquettes à neige")|(dftest["theme"] =="ski de randonnée")|(dftest["theme"] =="ski nordique")|(dftest["theme"] =="chalet")|(dftest["theme"] =="alpinisme")|(dftest["theme"] =="cascade de glace")|(dftest["theme"] =="raquette")|(dftest["theme"] =="logement privé"), "theme"] = "activités nordique"

for i in range(0, len(dftest)):
    if "rando nature" not in dftest.iloc[i]["theme"] and "cyclo" not in dftest.iloc[i]["theme"] and "activités nordique" not in dftest.iloc[i]["theme"] and "multi-activités" not in dftest.iloc[i]["theme"] and "ani-rando" not in dftest.iloc[i]["theme"]: 
        dftest.loc[i,"theme"] = "multi-activités"

region = ["auvergne", "rhone", "alpes", "pyrénées", "massif central", "sud-ouest", "provence", "bourogne", "bretagne", "normandie", "loire", "france", "occitanie", "normandes", "autres régions"]
pays_proche = ["madère", "maroc", "italie", "catalogne", "espagne", "grèce", "suisse", "ecosse", "irlande", "corse"]

for i in range(0, len(dftest)):
    for str in region:
        if str in dftest.loc[i,"lieux"]:
            dftest.loc[i, "lieux"] = "metropole"
    for str in pays_proche:
        if str in dftest.loc[i, "lieux"]:
            dftest.loc[i, "lieux"] = "dest_proche"
for i in range(0, len(dftest)):
    if dftest.loc[i, "lieux"] != "metropole" and dftest.loc[i, "lieux"] != "dest_proche" and dftest.loc[i, "lieux"] != "" :
        dftest.loc[i, "lieux"] = "dest_éloignée"


In [13]:
dftest["lieux"].value_counts()

metropole        251
dest_proche       81
dest_éloignée     81
                  10
Name: lieux, dtype: int64

Une fois le regroupement des modalités effectuées, nous effectuons une dernière phase de nettoyage. 
- Nous supprimons les séjours n'ayant pas de lieux (nous aurions pu les garder en affinant le scraping ou en utuilisant un tri à partir de la description)
- Nous supprimons les séjours ayant un prix égal à 0. Nous avions constaté lors de la phase de scraping, que certains séjours affichés n'était pas encore finalisé, et que le site n'indiquait donc pas de prix. Nous pensons qu'un des objectifs de notre phase d'apprentissage pourrait être d'estimé le prix de ces séjours dont l'organisateur n'a pas encore déterminé de prix. Il pourrait être intéressant de mettre ces 10 séjours de cotés. 

In [15]:
# gerer prix = 0 et destination vide 
# len(dftest[dftest["lieux"]==""]) + len(dftest[dftest["prix"]==0])
# dftest.drop(..., axis = 0)
# dftest[dftest["prix"]==0].index
dftest.drop(dftest[dftest["prix"]==0].index, axis = 0, inplace=True)
dftest.drop(dftest[dftest["lieux"]==""].index, axis = 0, inplace=True)


In [16]:
len(df) - len(dftest)

35

In [23]:
dftest = dftest.reset_index(drop=True)

In [24]:
dftest.dtypes

description      object
prix              int64
lieux          category
duree             int64
diff           category
theme          category
dtype: object

Nous transformons les variables **prix**, **lieux**, et **theme** en de `objet` à `category`afin de transformer ces chaînes de caractères en variables catégorielles.

In [19]:
dftest["lieux"] = dftest["lieux"].astype("category")
dftest["diff"] = dftest["diff"].astype("category")
dftest["theme"] = dftest["theme"].astype("category")

Enfin, nous sauvegardons le dataframe final en `.pkl`. Certaines descriptions issues du site "terre d'avenir" contiennent des caractères spéciaux. IL faudrait pa soucis de lisibilité les enlever. 

In [26]:
dftest.to_pickle("my_df.pkl")

In [27]:
dftest

,description,prix,lieux,duree,diff,theme
0,plaisirs neige &#224; sixt-fer-&#224;-cheval,690,metropole,8,Initié,multi-activités
1,haute clar&#233;e &quot;espace trappeur&quot;,860,metropole,7,Découverte,multi-activités
2,aventures hivernales sur le plateau de l&#39;a...,720,metropole,6,Découverte,multi-activités
3,neige et soleil du queyras en famille,730,metropole,7,Découverte,multi-activités
4,plaisirs neige dans les aravis,830,metropole,7,Découverte,multi-activités
5,vall&#233;e de la roy,670,metropole,7,Initié,multi-activités
6,davy crockett en haute bl&#233;one,730,metropole,6,Découverte,multi-activités
7,calanques et sensations proven&#231;ales,840,metropole,6,Initié,multi-activités
8,ea,760,metropole,7,Découverte,multi-activités
9,aventures canyon en sierra de guara,820,metropole,7,Découverte,multi-activités
